# Challenge: Análisis de reseñas de Amazon

¡Hola científica/científico de datos! Aquí tienes el challenge de esta clase.

Planteamiento del problema:
Imagina que trabajas en Amazon. Te han asignado a la división de análisis de datos de producto.

El trabajo de esta división es crucial, una de sus labores principales es recopilar los datos necesarios para implementar un sistema de recomendación de productos.

El equipo que implementará el modelo necesita que generemos el código para realizar 5 tareas de consulta. Prepara el código para que puedan realizar esas tareas:

1. Obtener el año en el que tuvimos más reseñas (tip: las fechas están como tipo texto, puedes utilizar `regex`).
2. Obtener únicamente las reseñas del año obtenido en el punto 1.
3. Filtrar las reseñas para quedarnos únicamente con las calificaciones más altas (overall: 5) y las más bajas (overall: 1).
4. Mediante un aggregation, obtener las reseñas "neutrales" (overall: 3) y añadir un campo extra al resultado con el formato: `label: "neutral"`. **NO ESCRIBAS EL VALOR EN LA BASE DE DATOS, SOLO DEBE APARECER EN EL RESULTADO DE LA CONSULTA**
5. Obtener el título (summary) y el nombre del autor (reviewerName), de las reseñas con mejores calificaciones (overall: 5).

## Instrucciones:

- Escribe el código para las tareas antes mencionadas.
- Utiliza la base de datos `master-data` presente en nuestro cluster y la colección `amazon`.
- Realiza cada tarea en un bloque diferente de código, es decir, no es necesario obtener un solo resultado, más bien, obtendrás un resultado distinto en cada tarea.

# Paquetes a utilizar

In [1]:
"""Mongo & Utilities"""
# Si quieres ejecutar este notebook localmente o te lanza error colab,
# puedes instalar las dependencias manualmente con los siguientes comandos
!pip install dnspython
!pip install pymongo[srv]
import dns
from pymongo import MongoClient
import pandas as pd
import json

Creación de la conexión

In [6]:
client = MongoClient('')

Creación de la base de datos `master-data` y de la colección `amazon`

In [8]:
nueva_bd = "master-data"
db = client[nueva_bd]
coll = db["amazon"]

Lectura del documento `amazon.json`

In [109]:
# Opening JSON file
with open('amazon.json') as json_file:
    amazonData = json.load(json_file)

print(amazonData[0])

{'_id': {'$oid': '60ac3be5a0f981616012e9cc'}, 'reviewerID': 'A2IBPI20UZIR0U', 'asin': '1384719342', 'reviewerName': 'cassandra tu "Yeah, well, that\'s just like, u...', 'helpful': [0, 0], 'reviewText': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,", 'overall': 5, 'summary': 'good', 'unixReviewTime': 1393545600, 'reviewTime': '02 28, 2014'}


Inserción de la colección a la base de datos

In [13]:
coll.insert_many(amazonData)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 52, 'errmsg': "_id fields may not contain '$'-prefixed fields: $oid is not valid for storage.", 'op': {'_id': {'$oid': '60ac3be5a0f981616012e9cc'}, 'reviewerID': 'A2IBPI20UZIR0U', 'asin': '1384719342', 'reviewerName': 'cassandra tu "Yeah, well, that\'s just like, u...', 'helpful': [0, 0], 'reviewText': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,", 'overall': 5, 'summary': 'good', 'unixReviewTime': 1393545600, 'reviewTime': '02 28, 2014'}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

An `error` appears due to the `$` symbol on the `_id` field, we will remove it using Pandas.

In [91]:
import pandas as pd

amazondf = pd.read_json('amazon.json')
amazondf.head()

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,{'$oid': '60ac3be5a0f981616012e9cc'},A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,{'$oid': '60ac3be5a0f981616012e9cd'},A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,{'$oid': '60ac3be5a0f981616012e9ce'},A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,{'$oid': '60ac3be5a0f981616012e9cf'},A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,{'$oid': '60ac3be5a0f981616012e9d0'},A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [92]:
def wrangler(row):
    """
    This function removes the '$' symbol from the key if the observation is a dictionary
    """
    if type(row) == dict:
        keys_values  = row.items()
        new_v = {str(key).replace('$', ''): str(value) for key, value in keys_values}

    return new_v

In [93]:
amazondf['_id'] = amazondf['_id'].map(lambda x: wrangler(x))
amazondf.head()

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,{'oid': '60ac3be5a0f981616012e9cc'},A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,{'oid': '60ac3be5a0f981616012e9cd'},A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,{'oid': '60ac3be5a0f981616012e9ce'},A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,{'oid': '60ac3be5a0f981616012e9cf'},A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,{'oid': '60ac3be5a0f981616012e9d0'},A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [126]:
amazon_clean = []

for i in amazondf.iterrows():
    row = i[1:][0]
    row_dict = row.to_dict()
    amazon_clean.append(row_dict)

amazon_clean[:5]

[{'_id': {'oid': '60ac3be5a0f981616012e9cc'},
  'reviewerID': 'A2IBPI20UZIR0U',
  'asin': '1384719342',
  'reviewerName': 'cassandra tu "Yeah, well, that\'s just like, u...',
  'helpful': [0, 0],
  'reviewText': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",
  'overall': 5,
  'summary': 'good',
  'unixReviewTime': 1393545600,
  'reviewTime': '02 28, 2014'},
 {'_id': {'oid': '60ac3be5a0f981616012e9cd'},
  'reviewerID': 'A14VAT5EAX3D9S',
  'asin': '1384719342',
  'reviewerName': 'Jake',
  'helpful': [13, 14],
  'reviewText': "The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old

In [127]:
coll.insert_many(amazon_clean)

Inserción exitosa de los documentos!

# Obtener el año en el que tuvimos más reseñas.



In [166]:
# Query for counting the number of documents per year

for i in list(range(2003, 2016)):

        query = {
                "reviewTime": {                
                "$regex": f'^.{{7}}{i}$',
                "$options" :'i' # case-insensitive
                }
                }
        results = coll.count_documents(query)
        print(f'Año: {i} --> Número de reseñas: {results}.') 

Año: 2003 --> Número de reseñas: 0.
Año: 2004 --> Número de reseñas: 6.
Año: 2005 --> Número de reseñas: 4.
Año: 2006 --> Número de reseñas: 8.
Año: 2007 --> Número de reseñas: 21.
Año: 2008 --> Número de reseñas: 43.
Año: 2009 --> Número de reseñas: 99.
Año: 2010 --> Número de reseñas: 262.
Año: 2011 --> Número de reseñas: 738.
Año: 2012 --> Número de reseñas: 1439.
Año: 2013 --> Número de reseñas: 2779.
Año: 2014 --> Número de reseñas: 1738.
Año: 2015 --> Número de reseñas: 0.


El año en que más se obtuvieron reseñas fue **2014**.

# Obtener únicamente las reseñas del año con más reseñas.



In [170]:
# Regex expression to retrieve only 2014 records
query = {
        "reviewTime": {        
        "$regex": '^.{7}2014$'
        }
        }

results = coll.find(query)

# List for storing the results
reseñas2014 = [] 

for reseña in results:
        reseñas2014.append(reseña)

# Showing only first two documents
reseñas2014[:2] 


[{'_id': {'oid': '60ac3be5a0f981616012e9cc'},
  'reviewerID': 'A2IBPI20UZIR0U',
  'asin': '1384719342',
  'reviewerName': 'cassandra tu "Yeah, well, that\'s just like, u...',
  'helpful': [0, 0],
  'reviewText': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",
  'overall': 5,
  'summary': 'good',
  'unixReviewTime': 1393545600,
  'reviewTime': '02 28, 2014'},
 {'_id': {'oid': '60ac3be5a0f981616012e9cf'},
  'reviewerID': 'A2C00NNG1ZQQG2',
  'asin': '1384719342',
  'reviewerName': 'RustyBill "Sunday Rocker"',
  'helpful': [0, 0],
  'reviewText': 'Nice windscreen protects my MXL mic and prevents pops. Only thing is that the gooseneck is only marginally able to hold the screen in position and requires careful positioning of the clamp to avoid sagging.',
  'overall': 5,
 

# Filtrar las reseñas para quedarnos únicamente con las calificaciones más altas y las más bajas.

In [186]:
# Query for retrieving only the reviews with the highest and the lowest scores from year 2014
query = {
        "reviewTime": {        
        "$regex": '^.{7}2014$'
        },
        "$or": [{"overall": 5}, {"overall": 1}]
        }

results = coll.find(query)

# List for storing the reviews with the highest and the lowest scores
cal_altas_bajas = [] 

for reseña in results:
        cal_altas_bajas.append(reseña)

# Length of the retrieved documents
len(cal_altas_bajas)


1222

# Mediante un aggregation, obtener las reseñas "neutrales" y etiquetarlas como "neutral" en el resultado de la consulta.

In [189]:
# Aggregation Function
neutral_reviews = db.amazon.aggregate(
    [
      {
          '$match': {
              'overall': 3              
          }
      }, {
          '$addFields': {
              'label': 'neutral'
          }
      }]

)

# Displaying first three results
print(list(neutral_reviews)[:3])

[{'_id': {'oid': '60ac3be5a0f981616012e9d3'}, 'reviewerID': 'AJNFQI3YR6XJ5', 'asin': 'B00004Y2UT', 'reviewerName': 'Fender Guy "Rick"', 'helpful': [0, 0], 'reviewText': "I now use this cable to run from the output of my pedal chain to the input of my Fender Amp. After I bought Monster Cable to hook up my pedal board I thought I would try another one and update my guitar. I had been using a high end Planet Waves cable that I bought in the 1980's... Once I found out the input jacks on the new Monster cable didn't fit into the Fender Strat jack I was a little disappointed... I didn't return it and as stated I use it for the output on the pedal board. Save your money... I went back to my Planet Waves Cable...I payed $30.00 back in the eighties for the Planet Waves which now comes in at around $50.00. What I'm getting at is you get what you pay for. I thought Waves was a lot of money back in the day...but I haven't bought a guitar cable since this one...20 plus years and still working...Pla

# Obtener el título y el nombre del autor, de las reseñas con mejores calificaciones.

In [193]:
# Query for retrieving the reviews with the highest scores from year 2014
query = {
        "reviewTime": {        
        "$regex": '^.{7}2014$'
        },
        "overall": 5
        }

results = coll.find(query)

# List for storing the title and reviewer Name of the reviews with the highest scores
title_name = [] 

for reseña in results:
        title_name.append({'Title': reseña['summary'], 'Name' : reseña['reviewerName']}) # Title and name of the reviewer as a dict

# First 5 results
title_name[:5]


[{'Title': 'good',
  'Name': 'cassandra tu "Yeah, well, that\'s just like, u...'},
 {'Title': 'GOOD WINDSCREEN FOR THE MONEY',
  'Name': 'RustyBill "Sunday Rocker"'},
 {'Title': 'No more pops when I record my vocals.', 'Name': 'SEAN MASLANKA'},
 {'Title': "Monster Standard 100 - 21' Instrument Cable", 'Name': 'Brian'},
 {'Title': 'Fender 18 Feet California Clear Instrument Cable - Lake Placid Blue',
  'Name': 'Ricky Shows'}]

In [ ]:
#End